<a href="https://colab.research.google.com/github/huluobohua/polycrisis/blob/main/Coding_data_transformations_UNICEFRegions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://github.com/huluobohua/polycrisis/blob/main/0.%20Polycrisis%20dashboard%20-%20Raw%20data%20for%20Mariana2.xlsx?raw=true

# Coding data transformations

First, let's mount Google Drive so we can load files from Google Drive Folders.

In [ ]:
#/* from google.colab import drive
#drive.mount('/content/gdrive')
#%cd /content/gdrive/'Shared with me'/'Unicef Foresight and Policy' */

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: '/content/gdrive/Shared with me/Unicef Foresight and Policy/'
/content


Then, we'll load the file that contains the data to be dispayed.

In [ ]:
#Locate file
import pandas as pd
import openpyxl
f = 'https://github.com/huluobohua/polycrisis/blob/main/0.%20Polycrisis%20dashboard%20-%20Raw%20data%20for%20Mariana2.xlsx?raw=true'


### GBP indicator B

In [ ]:
#Load xlsx sheet
import numpy as np
import pandas as pd
df1 = pd.read_excel(f,
  sheet_name="GDP indicator B",
  skiprows=1,
  nrows=197,
  usecols="A:K"
);

df1.head()

/usr/local/lib/python3.8/dist-packages/pandas/io/excel/_base.py:1272: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return self._reader.parse(


,ISO Code,IMF growth forecast for 2023 in Oct 21,IMF growth forecast for 2023 in Oct 22,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme
0,ABW,6.200,1.950,ABW,Aruba,Latin America and the Caribbean,NaN,0,0
1,AFG,NaN,NaN,AFG,Afghanistan,Southern Asia,SA,0,1
2,AGO,3.328,3.427,AGO,Angola,Sub-Saharan Africa,ESA,0,1
3,AIA,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1
4,ALB,4.100,2.500,ALB,Albania,Southern Europe,ECA,0,1


In [6]:
#Replicate column C using A1 and A2
df1[a1_gdpb] = df1["IMF growth forecast for 2023 in Oct 21"]
df1[a2_gdpb] = df1["IMF growth forecast for 2023 in Oct 22"]
df1[c_gdpb] = 0

df1_c = ( df1[a2_gdpb] - df1[a1_gdpb] ) / abs(df1[a1_gdpb]) #operation


df1_c.head()

0   -0.685484
1         NaN
2    0.029748
3         NaN
4   -0.390244
dtype: float64

In [8]:
#Replicate column D using C

df1_d = df1_c + abs( df1_c.min() ) #operation

df1_d.head()

0    1.454516
1         NaN
2    2.169748
3         NaN
4    1.749756
dtype: float64

In [9]:
#Replicate column I using D

#normalisation
q1, q3 = df1_d.quantile(.25), df1_d.quantile(.75)
outlier_uppercut = q3 + 2*(q3-q1)
filter = df1_d.between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((d-df1_d.min())/(df1_d[filter].max()-df1_d.min()) if d < outlier_uppercut
     else (1. if not np.isnan(d) else np.nan))
     for d in df1_d 
])

#reindexing
df1_i = 10 - 10*(normalisation)
df1_i.head()

0    4.832758
1         NaN
2    2.291863
3         NaN
4    3.783903
dtype: float64

### Food inflation indicator

In [10]:
#Load xlsx sheet
df2 = pd.read_excel(f,
  sheet_name="Food inflation indicator",
  skiprows=1,
  nrows=197,
  usecols="A:J"
);

df2.head()

,Country,ISO,Food inflation,Food inflation shifted right,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme
0,Aruba,ABW,11.53,12.51,ABW,Aruba,Latin America and the Caribbean,NaN,0,0
1,Afghanistan,AFG,17.59,18.57,AFG,Afghanistan,Southern Asia,SA,0,1
2,Angola,AGO,16.80,17.78,AGO,Angola,Sub-Saharan Africa,ESA,0,1
3,Anguila,AIA,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1
4,Albania,ALB,15.30,16.28,ALB,Albania,Southern Europe,ECA,0,1


In [16]:
#Replicate column B using A
a_fii = "Food inflation"
b_fii = "Food inflation shifted right"

df2_b = df2[a_fii] + abs(df2[a_fii].min()) #operation
assert all( df2_b.dropna() == df2[b_fii].dropna() ) #sanity check

df2_b.head()

0    12.51
1    18.57
2    17.78
3      NaN
4    16.28
Name: Food inflation, dtype: float64

In [17]:
#Replicate column I*10 using B
i_fii = "Normalized*10"

#normalisation
q1, q3 = df2_b.quantile(.25), df2_b.quantile(.75)
outlier_uppercut = q3 + 2*(q3-q1) 
#q3 + 2*(q3-q1) there is a manual override in the sheet
filter = df2_b.between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((b-df2_b.min())/(df2_b[filter].max()-df2_b.min()) if b < outlier_uppercut
     else (1. if not np.isnan(b) else np.nan))
     for b in df2_b 
])

#reindexing
df2_i = 10*(normalisation)
df2_i

0      2.601913
1      3.862313
2      3.698003
3           NaN
4      3.386023
         ...   
192         NaN
193    9.993482
194    2.699667
195    7.850501
196    8.390537
Length: 197, dtype: float64

### Credit Score indicator

In [19]:
#Load xlsx sheet
df3 = pd.read_excel(f,
  sheet_name="Credit Score indicator",
  skiprows=1,
  nrows=197,
  usecols="A:G"
);

df3.head()

,Country,ISO,Credit rating average,Credit rating average inverted (to make more positive worse),Credit rating average normalized * 10 (values in red are imputted - exclude from indexation range),ImputedIMF4Levels,ISO3
0,Aruba,ABW,11.5,9.5,4.523810,NaN,ABW
1,Afghanistan,AFG,NaN,NaN,7.500000,1.0,AFG
2,Angola,AGO,6.0,15.0,7.142857,NaN,AGO
3,Albania,AIA,NaN,NaN,NaN,NaN,AIA
4,Andorra,ALB,8.0,13.0,6.190476,NaN,ALB


In [20]:
#Replicate column B using A
a_csi = "Credit rating average"
b_csi = "Credit rating average inverted (to make more positive worse)"

df3_b = 21. - df3[a_csi] #operation
assert all( df3_b.dropna() == df3[b_csi].dropna() ) #sanity check

df3_b.head()

0     9.5
1     NaN
2    15.0
3     NaN
4    13.0
Name: Credit rating average, dtype: float64

In [21]:
#Replicate column I*10 using B
i_csi = "Credit rating average normalized * 10 (values in red are imputted - exclude from indexation range)"

#normalisation
q1, q3 = df3[a_csi].quantile(.25), df3[a_csi].quantile(.75) #using column A
# q1, q3 = df3_b.quantile(.25), df3_b.quantile(.75) #using column B
outlier_uppercut = q3 + 2*(q3-q1)
filter = df3_b.between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((b-df3_b.min())/(df3_b[filter].max()-df3_b.min()) if b < outlier_uppercut
     else (1. if not np.isnan(b) else np.nan))
     for b in df3_b 
])

#reindexing
df3_i = 10*(normalisation)
df3_i

0      4.523810
1           NaN
2      7.142857
3           NaN
4      6.190476
         ...   
192         NaN
193         NaN
194    5.555556
195    9.047619
196    9.523810
Length: 197, dtype: float64

### External conflict risk

In [22]:
#Load xlsx sheet
df4 = pd.read_excel(f,
  sheet_name="External conflict risk",
  skiprows=1,
  nrows=197,
  usecols="A:G"
);

df4.head()

,ISO3,GPI Militarization 2022,GPI Relationship with neighbors 2022,Current external conflict (5 if current; 3 if probable or stand-off; 0 if none),GPI External Components,CountryName,UNSubRegionName
0,ABW,NaN,NaN,0.0,NaN,Aruba,Latin America and the Caribbean
1,AFG,2.472,4.336,0.0,2.269333,Afghanistan,Southern Asia
2,AGO,1.706,3.345,0.0,1.683667,Angola,Sub-Saharan Africa
3,AIA,NaN,NaN,NaN,NaN,Anguila,Latin America and the Caribbean
4,ALB,1.666,2.899,0.0,1.521667,Albania,Southern Europe


In [23]:
#Replicate column C123 using A1, A2  and A3
a1_ecr = "GPI Militarization 2022"
a2_ecr = "GPI Relationship with neighbors 2022"
a3_ecr = "Current external conflict (5 if current; 3 if probable or stand-off; 0 if none)"
c123_ecr = "GPI External Components"

df4_c123 = ( df4[a1_ecr]+df4[a2_ecr]+df4[a3_ecr] )/3 #operation
assert all( df4_c123.dropna() == df4[c123_ecr].dropna() ) #sanity check

df4_c123.head()

0         NaN
1    2.269333
2    1.683667
3         NaN
4    1.521667
dtype: float64

In [24]:
#Replicate column I using C123
i_ecr = "GPI external components normalized"

#normalisation
q1, q3 = df4_c123.quantile(.25), df4_c123.quantile(.75) #using column C123
outlier_uppercut = q3 + 2*(q3-q1)
filter = df4_c123.between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((c-df4_c123.min())/(df4_c123[filter].max()-df4_c123.min()) if c < outlier_uppercut
     else (1. if not np.isnan(c) else np.nan))
     for c in df4_c123 
])

#reindexing
df4_i = 10*(normalisation)
df4_i

0           NaN
1      6.827930
2      3.906899
3           NaN
4      3.098919
         ...   
192         NaN
193    6.947631
194    2.239401
195    4.036575
196    5.586035
Length: 197, dtype: float64

### Political stability abs conf

In [42]:
#Load xlsx sheet
df5 = pd.read_excel(f,
  sheet_name="Political stability abs conf",
  skiprows=1,
  nrows=197,
  usecols="A:J"
)

df5.head()

,ISO Code_orig,WGI (political stability),2021 WGI Political stability,WGI normalized,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme
0,ABW,0.364940,1.474335,0.364940,ABW,Aruba,Latin America and the Caribbean,NaN,0,0
1,AFG,9.644537,-2.529855,9.644537,AFG,Afghanistan,Southern Asia,SA,0,1
2,AGO,5.429013,-0.710836,5.429013,AGO,Angola,Sub-Saharan Africa,ESA,0,1
3,AIA,NaN,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1
4,ALB,3.528032,0.109446,3.528032,ALB,Albania,Southern Europe,ECA,0,1


In [43]:
#Replicate column I using A
a_psac = "2021 WGI Political stability"
i_psac = "WGI normalized"

#normalisation
q1, q3 = df5[a_psac].quantile(.25), df5[a_psac].quantile(.75) #using column A
outlier_uppercut = q3 + 2*(q3-q1)
filter = df5[a_psac].between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((a-df5[a_psac].min())/(df5[a_psac][filter].max()-df5[a_psac].min()) if a < outlier_uppercut
     else (1. if not np.isnan(a) else np.nan))
     for a in df5[a_psac]
])

#reindexing
df5_i = 10 - 10*(normalisation)
df5_i

0      0.364940
1      9.644537
2      5.429013
3           NaN
4      3.528032
         ...   
192    1.209468
193    9.779374
194    5.419142
195    3.642120
196    6.161210
Length: 197, dtype: float64

### Climate indicator

In [40]:
#Load xlsx sheet
df6 = pd.read_excel(f,
  sheet_name="Climate indicator",
  skiprows=1,
  nrows=197,
  usecols="A:K"
);

df6.head()

,ISO,ISO Code,ND-GAIN Climate Vulnerability Index,ISO Code.1,Climate index taken to 10 point scale,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme
0,ABW,ABW,NaN,ABW,NaN,ABW,Aruba,Latin America and the Caribbean,NaN,0,0
1,AFG,AFG,0.579128,AFG,7.718004,AFG,Afghanistan,Southern Asia,SA,0,1
2,AGO,AGO,0.505320,AGO,5.959143,AGO,Angola,Sub-Saharan Africa,ESA,0,1
3,AIA,AIA,NaN,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1
4,ALB,ALB,0.414296,ALB,3.789999,ALB,Albania,Southern Europe,ECA,0,1


In [41]:
#Replicate column I using A
a_ci = "ND-GAIN Climate Vulnerability Index"
i_ci = "Climate index taken to 10 point scale"

#normalisation
q1, q3 = df6[a_ci].quantile(.25), df6[a_ci].quantile(.75)
outlier_uppercut = q3 + 2*(q3-q1)
q1,q3,outlier_uppercut
filter = df6[a_ci].between(q1, outlier_uppercut, inclusive="both")
normalisation = pd.Series([
    #operation
     ((a-df6[a_ci].min())/(df6[a_ci][filter].max()-df6[a_ci].min()) if a < outlier_uppercut
     else (1. if not np.isnan(a) else np.nan))
     for a in df6[a_ci]
])

#reindexing
df6_i = 10*(normalisation)
df6_i

0           NaN
1      7.718004
2      5.959143
3           NaN
4      3.789999
         ...   
192    5.525553
193    7.222984
194    3.811904
195    6.331208
196    7.118473
Length: 197, dtype: float64

In [44]:
df1.head()


,ISO Code,IMF growth forecast for 2023 in Oct 21,IMF growth forecast for 2023 in Oct 22,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme,Relative IMF growth projection
0,ABW,6.200,1.950,ABW,Aruba,Latin America and the Caribbean,NaN,0,0,0
1,AFG,NaN,NaN,AFG,Afghanistan,Southern Asia,SA,0,1,0
2,AGO,3.328,3.427,AGO,Angola,Sub-Saharan Africa,ESA,0,1,0
3,AIA,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1,0
4,ALB,4.100,2.500,ALB,Albania,Southern Europe,ECA,0,1,0


In [45]:
df2.head()

,Country,ISO,Food inflation,Food inflation shifted right,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme,...,103.98,23.68,36.08,12.26,158.98,3.77,48.04866,12.98,37.74521,40.341699999999996
0,Aruba,ABW,11.53,12.51,ABW,Aruba,Latin America and the Caribbean,NaN,0,0,...,103.98,23.68,36.08,12.26,158.98,3.77,48.04866,12.98,37.74521,40.3417
1,Afghanistan,AFG,17.59,18.57,AFG,Afghanistan,Southern Asia,SA,0,1,...,103.98,23.68,36.08,12.26,158.98,3.77,48.04866,12.98,37.74521,40.3417
2,Angola,AGO,16.80,17.78,AGO,Angola,Sub-Saharan Africa,ESA,0,1,...,103.98,23.68,36.08,12.26,158.98,3.77,48.04866,12.98,37.74521,40.3417
3,Anguila,AIA,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1,...,103.98,23.68,36.08,12.26,158.98,3.77,48.04866,12.98,37.74521,40.3417
4,Albania,ALB,15.30,16.28,ALB,Albania,Southern Europe,ECA,0,1,...,103.98,23.68,36.08,12.26,158.98,3.77,48.04866,12.98,37.74521,40.3417


In [46]:
df3.head()


,Country,ISO,Credit rating average,Credit rating average inverted (to make more positive worse),Credit rating average normalized * 10 (values in red are imputted - exclude from indexation range),ImputedIMF4Levels,ISO3
0,Aruba,ABW,11.5,9.5,4.523810,NaN,ABW
1,Afghanistan,AFG,NaN,NaN,7.500000,1.0,AFG
2,Angola,AGO,6.0,15.0,7.142857,NaN,AGO
3,Albania,AIA,NaN,NaN,NaN,NaN,AIA
4,Andorra,ALB,8.0,13.0,6.190476,NaN,ALB


In [47]:
df4.head()


,ISO3,GPI Militarization 2022,GPI Relationship with neighbors 2022,Current external conflict (5 if current; 3 if probable or stand-off; 0 if none),GPI External Components,CountryName,UNSubRegionName
0,ABW,NaN,NaN,0.0,NaN,Aruba,Latin America and the Caribbean
1,AFG,2.472,4.336,0.0,2.269333,Afghanistan,Southern Asia
2,AGO,1.706,3.345,0.0,1.683667,Angola,Sub-Saharan Africa
3,AIA,NaN,NaN,NaN,NaN,Anguila,Latin America and the Caribbean
4,ALB,1.666,2.899,0.0,1.521667,Albania,Southern Europe


In [48]:
df5.head()

,ISO Code_orig,WGI (political stability),2021 WGI Political stability,WGI normalized,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme
0,ABW,0.364940,1.474335,0.364940,ABW,Aruba,Latin America and the Caribbean,NaN,0,0
1,AFG,9.644537,-2.529855,9.644537,AFG,Afghanistan,Southern Asia,SA,0,1
2,AGO,5.429013,-0.710836,5.429013,AGO,Angola,Sub-Saharan Africa,ESA,0,1
3,AIA,NaN,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1
4,ALB,3.528032,0.109446,3.528032,ALB,Albania,Southern Europe,ECA,0,1


In [49]:
df6.head()

,ISO,ISO Code,ND-GAIN Climate Vulnerability Index,ISO Code.1,Climate index taken to 10 point scale,ISO3,CountryName,UNSubRegionName,UNICEFRegionCode,NATCOM,Programme
0,ABW,ABW,NaN,ABW,NaN,ABW,Aruba,Latin America and the Caribbean,NaN,0,0
1,AFG,AFG,0.579128,AFG,7.718004,AFG,Afghanistan,Southern Asia,SA,0,1
2,AGO,AGO,0.505320,AGO,5.959143,AGO,Angola,Sub-Saharan Africa,ESA,0,1
3,AIA,AIA,NaN,NaN,NaN,AIA,Anguila,Latin America and the Caribbean,CAO,0,1
4,ALB,ALB,0.414296,ALB,3.789999,ALB,Albania,Southern Europe,ECA,0,1
